In [56]:
from utils.converter import pdf_converter
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

In [2]:
array_list = pdf_converter("documents/", min_length=200)

In [3]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

docs = array_list
doc_emb = model.encode(docs)

In [4]:
doc_emb.shape,len(docs)

((300, 384), 300)

In [5]:


#Load the model


#Encode query and documents


def compute_score(doc_emb, query_emb, top_k=3):
    #Compute dot score between query and all document embeddings
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    return [docs[i] for i in np.argsort(scores)[-5:]]
    #Combine docs & scores
#     doc_score_pairs = list(zip(docs, scores))
    #Sort by decreasing score
#     doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
#     return doc_score_pairs
#     for doc, score in doc_score_pairs[0:5]:
#         print(score, doc)
#     return [doc for doc,score in doc_score_pairs[:top_k]]
#     return doc_score_pairs
#Output passages & scores



In [34]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name ="deepset/roberta-base-squad2-distilled" #
# model_name="deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

# a) Get predictions


In [67]:

def qa_model(query, relevent_docs):
    QA_input = [{
        'question': query,#'Why is model conversion important?',
        'context': doc#'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
    } for doc in relevent_docs]
    res = nlp(QA_input)
    return res

# b) Load model & tokenizer
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
def get_answer(query):
    query_emb = model.encode(query)
    relevent_docs = compute_score(doc_emb, query_emb,top_k=5)
    answer = qa_model(query, relevent_docs)
    __  = {key.update({"paragraph":relevent_docs.pop(0), "score":round(key["score"], 3)}) for key in answer }
    answer_df = pd.DataFrame(answer)
    return answer_df.sort_values(["score"],ascending= False)

In [68]:
# for doc in doc_score_pairs[0:5]:
#     print(doc)
temp = get_answer("where is he born")

In [69]:
temp

,score,start,end,answer,paragraph
0,0.505,48,57,Elanthoor,Mohanlal Viswanathan was born in the village o...
3,0.412,4,24,Mohanlal Birth Place,24. Mohanlal Birth Place Archived 25 April 2...
4,0.043,5,25,Mohanlal Viswanathan,Born Mohanlal Viswanathan[1][2]
1,0.006,10,19,Ujala Ali,"17. Khan, Ujala Ali (14 September 2013). ""Reig..."
2,0.000,11,12,.,Relatives K. Balaji (father-in-law)


In [70]:
# query = "which district was  he born"
# query_emb = model.encode(query)

# relevent_docs = compute_score(doc_emb, query_emb,top_k=5)
# relevent_docs

In [21]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ["""No matter how meticulous the plan, disruptors are more than likely to appear at some \
point during large-scale transformations. Careful planning is no substitute for flexibility, \
so the program management approach needs in-built resilience and adaptability. When \
a hurdle appears, the teams need to adjust quickly, refocus, and be open to adopting \
an incremental micro delivery approach when an initial macro delivery was originally \ 
planned, for example. With the right approach and mindset, disruptions don’t necessarily \
have to result in extending key delivery timelines.No matter how meticulous the plan, disruptors are more than likely to appear at some \
point during large-scale transformations. Careful planning is no substitute for flexibility, \
so the program management approach needs in-built resilience and adaptability. When \
a hurdle appears, the teams need to adjust quickly, refocus, and be open to adopting \
an incremental micro delivery approach when an initial macro delivery was originally \ 
planned, for example. With the right approach and mindset, disruptions don’t necessarily \
have to result in extending key delivery timelines. No matter how meticulous the plan, disruptors are more than likely to appear at some \
point during large-scale transformations. Careful planning is no substitute for flexibility, \
so the program management approach needs in-built resilience and adaptability. When \
a hurdle appears, the teams need to adjust quickly, refocus, and be open to adopting \
an incremental micro delivery approach when an initial macro delivery was originally \ 
planned, for example. With the right approach and mindset, disruptions don’t necessarily \
have to result in extending key delivery timelines.No matter how meticulous the plan, disruptors are more than likely to appear at some \
point during large-scale transformations. Careful planning is no substitute for flexibility, \
so the program management approach needs in-built resilience and adaptability. When \
a hurdle appears, the teams need to adjust quickly, refocus, and be open to adopting \
an incremental micro delivery approach when an initial macro delivery was originally \ 
planned, for example. With the right approach and mindset, disruptions don’t necessarily \
have to result in extending key delivery timelines."""]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)



Token indices sequence length is longer than the specified maximum sequence length for this model (921 > 512). Running this sequence through the model will result in indexing errors


In [22]:
encoded_input["input_ids"][0].shape

torch.Size([921])

In [27]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=False, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)


Token indices sequence length is longer than the specified maximum sequence length for this model (921 > 512). Running this sequence through the model will result in indexing errors


In [28]:
encoded_input["input_ids"].shape

torch.Size([1, 921])